In [174]:
import os
import re
from glob import glob
import pandas as pd

def clean_folder(directory):

    file_read = 0
    file_rename_success = 0
    file_rename_failed = 0
    file_rename_NoNeed = 0
    
    # Traverse through the directory
    for root, _, files in os.walk(directory):
        for file in files:
            file_read += 1
            # if file=='tips 62d1702502fe49ef9e4522aeab3474e1.md':
            old_file_path = os.path.join(root, file)

            file_name_path=os.path.splitext(old_file_path)[0]
            extension=os.path.splitext(old_file_path)[1]

            file_name=os.path.split(file_name_path)[1]
            print('Current file:', file)
            print('    file_name: ',file_name)
            print('    Extension:',extension)

            #check if we have hashcode after filename (space+32 characters for hash)
            if re.match('.+\s\w{32}', file_name):
                Rename_flag = True
                print('need to remove ID: Yes')
                new_file_name = file_name[:-33]
                updated_path= os.path.join(root,new_file_name)+extension

                # Check if the file exists before renaming
                if os.path.exists(updated_path):
                    # If the new path already exists, add a number suffix
                    i = 2
                    while os.path.exists(updated_path):
                        new_file_name = f"{new_file_name} ({i})"
                        updated_path = os.path.join(root, new_file_name)+extension
                        i += 1
                os.rename(old_file_path,updated_path)
                
                if os.path.exists(updated_path):
                    print('Rename success!')
                    file_rename_success +=1 
                else:
                    print('Rename Fail!')
                    file_rename_failed +=1
                
            else:
                print('need to remove ID: No')
                updated_path= old_file_path
                Rename_flag = False
                file_rename_NoNeed += 1 
            print(f'old_file_path= r"{old_file_path}"',)
            print(f'updated_path= r"{updated_path}"',)
            if not Rename_flag:
                print('No need to rename')
            

            print('\n','*'*25)
    return file_read,file_rename_success,file_rename_NoNeed,file_rename_failed




Current file: After the project.md
    file_name:  After the project
    Extension: .md
need to remove ID: No
old_file_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\After the project.md"
updated_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\After the project.md"
No need to rename

 *************************
Current file: Alteryx.md
    file_name:  Alteryx
    Extension: .md
need to remove ID: No
old_file_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\Alteryx.md"
updated_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\Alteryx.md"
No need to rename

 *************************
Current file: Analytic palette 103.csv
    file_name:  Analytic palette 103
    Extension: .csv
need to remove ID: No
old_file_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\Analytic palette 103.csv"
updated_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca

In [188]:
def list_subfolder(folder) -> list:
    subfolders = []
    for root, dirs, files in os.walk(folder):
        for dir_name in dirs:
            subfolders.append(os.path.join(root, dir_name))
    return subfolders

# create a df containing the folder path and level of hierarchy - starting from the deepest
def clear_folder(folder):
    subfolders = list_subfolder(folder)

    #record the stat
    folders_read=len(subfolders)

    subfolders = list_subfolders(directory_path)
    length_list=[len(sub.split('\\')) for sub in subfolders]
    max_length=max(length_list)
    df = pd.DataFrame(list(zip(subfolders, length_list)),
                columns =['subfolders', 'length'])
    df=df.sort_values(['length'],ascending=False)

    df[['root', 'folder_name']] = df['subfolders'].apply(lambda x: pd.Series(os.path.split(x)))

    # Define a function to check if we have hashcode after foldername (space+32 characters for hash)
    def check_flag(folder_name):
        return bool(re.match('.+\s\w{32}', folder_name))

    # Apply the function to 'folder_name' column to create the 'Flag' column
    df['Flag'] = df['folder_name'].apply(check_flag)

    #record the stat
    folders_rename_NoNeed = len(df[df['Flag'] == False])

    #filter the folder df for those with hash code
    df_clean= df[df['Flag'] == True]
    df_clean.reset_index(inplace=True,drop=True)
    df_clean

    # #test case
    # df_clean=df_clean.head(3)

    #record stat
    folders_rename_success = 0
    folders_rename_fail = 0

    # rename the folder in df row by row
    for index,row in df_clean.iterrows():
        old_path=row['subfolders']
        new_path=old_path[:-33]

        # Check if the directory exists before renaming
        if os.path.exists(new_path):
            # If the new path already exists, add a number suffix
            i = 2
            while os.path.exists(new_path):
                new_path = f"{new_path} ({i})"
                i += 1

        print(f'old_path=r"{old_path}"')
        print(f'new_path=r"{new_path}"')

        try:
            os.rename(old_path,new_path)
        except Exception as e:
            print(f'Error occur: {e}')
        if os.path.exists(new_path):
            folders_rename_success += 1
        else:
            folders_rename_fail += 1
        
        print('*'*25)

    return folders_read,folders_rename_NoNeed,folders_rename_success,folders_rename_fail

# usage - input a directory path
directory_path = r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c"

file_read,file_rename_success,file_rename_NoNeed,file_rename_failed = clean_folder(directory_path)
folders_read,folders_rename_NoNeed,folders_rename_success,folders_rename_fail = clear_folder(directory_path)
print(f"A total of {file_read} files were read.\n     {file_rename_NoNeed} doesn't need to be renamed\n     {file_rename_success} renamed success\n     {file_rename_failed} renamed failed\n")
print(f"A total of {folders_read} folders were read:\n     {folders_rename_NoNeed} doesn't need to be renamed\n     {folders_rename_success} renamed success\n     {folders_rename_fail} renamed failed")

Current file: After the project.md
    file_name:  After the project
    Extension: .md
need to remove ID: No
old_file_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\After the project.md"
updated_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\After the project.md"
No need to rename

 *************************
Current file: Alteryx.md
    file_name:  Alteryx
    Extension: .md
need to remove ID: No
old_file_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\Alteryx.md"
updated_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\Alteryx.md"
No need to rename

 *************************
Current file: Analytic palette 103.csv
    file_name:  Analytic palette 103
    Extension: .csv
need to remove ID: No
old_file_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca865deb4b53652d6c\Analytic palette 103.csv"
updated_path= r"C:\Users\StanleyChan\Downloads\Note 2a556892b10b4bca

In [189]:
subfolders=list_subfolder(directory_path)
subfolders

['C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\Analytic palette',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\AWS',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\blog IDEA',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\Business email template',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\Client Project',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\Client research',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\CV after DS',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\Data',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\DBT',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\Financial knowledge',
 'C:\\Users\\StanleyChan\\Downloads\\Note 2a556892b10b4bca865deb4b53652d6c\\Github',
 'C:\\Us